# "EarthAnnotator"

## Prototype web-based image labeling tool

Daniel Buscombe, daniel.buscombe@nau.edu

October 2018

Implements the technique outlined by [Buscombe & Ritchie (2018)](https://www.mdpi.com/2076-3263/8/7/244)

### 1. Load libraries

In [ ]:
from PIL import Image ##from imageio import imread
import numpy as np
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews.streams import FreehandDraw
hv.extension('bokeh')

In [ ]:
from funcs.crf_utils import *
from funcs.widgets_utils import *
from funcs.file_select import FileBrowser
from funcs.tile_utils import sliding_window
global labels_widget

In [ ]:
import panel, param
from bokeh.plotting import figure
from panel.layout import *
from panel.widgets import *
panel.extension()

### 2. Pick an image 

In [ ]:
#imfile = '/home/jovyan/EarthAnnotator/data/NewRiver_worldImageryRGB_20m.tif'

In [ ]:
file_picker = FileBrowser()
file_picker.widget()

In [ ]:
imfile = file_picker.path

In [ ]:
im = Image.open(imfile)
im = im.convert("RGB")
nx, ny, nz = np.shape(im)

### 3. Make labels and colors

#### Instructions:
1. Create class labels and assign each label a color

In [ ]:
label_editor, labels_widget = create_colorpicker()
label_editor

In [ ]:
labels, colors = get_labels_and_colors(label_editor)

In [ ]:
labels_widget = panel.widgets.Select(options=labels)

In [ ]:
brush = 5

### 4. Freehand Drawing

#### Instructions:
1. Choose a label from the dropdown menu (and optionally choose a line width)
2. Click the 'pen' tool, and freehand draw on image
3. When done with freehand draw for each label, click "Done with Label"
4. Choose next label and repeat steps 1-3.

In [ ]:
class EarthAnnotate(param.Parameterized):
    
    label = param.ObjectSelector(objects=labels, default=labels[0])
    anno = []
    done_with_label = param.Action(lambda x: x.anno.append(np.column_stack(access_annotation_coordinates(x.freehand_stream))),
                                  precedence=1.0)
    
    def make_view(self, **kwargs):
        color_index = labels.index(self.label)
        opts = dict(line_width=brush, color=colors[color_index], width=400, height=600)
        path = hv.Path([[(0, 0), (0, 0)]]).options(**opts)
        self.freehand_stream = FreehandDraw(source=path, num_objects=999)
        bounds=(0,0,ny,nx)   # Coordinate system: (left, bottom, top, right)
        img = hv.RGB(np.array(im), bounds=bounds) #hv.Image(im, bounds=bounds)

        return img * path 

e_anno = EarthAnnotate(name="Image Annotation")
panel.Row(e_anno.make_view, e_anno)

In [ ]:
anno = dict(zip(labels, e_anno.anno))

### 5. Choose parameters and run pixel estimation model

In [ ]:
# This parameter penalizes small pieces of segmentation that are
# spatially isolated -- enforces more spatially consistent segmentations
compat_spat=12 ##non-dimensional
# larger values = larger pieces of segmentation allowed

In [ ]:
# This parameter penalizes pieces of segmentation that are
# less uniform in color -- enforces more consistent segmentations in colorspace
compat_col=100 ##non-dimensional
# larger values = pieces of segmentation with less similar image intesity allowed

In [ ]:
# Scaling parameters: tolerances in intensity and location
theta=60 
# larger values = pixel pairs can be considered to be the same class label with less similar location/intensity

In [ ]:
# number of iterations for algorithm (generally, larger the better, but only to a point)
n_iter=15

In [ ]:
Lc = get_sparse_label(anno, nx, ny, labels, brush)
res = getCRF(im, Lc, theta, n_iter, labels, compat_spat, compat_col)
out = get_rgb(res, labels, colors)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(im)
plt.imshow(out, alpha=0.5);

### 6. Write image to file

In [ ]:
write_label_image(imfile, out)